# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Elemento de lista
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [2]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
import os
import requests
output_dir = "audios_fabulas_mp3"
os.makedirs(output_dir, exist_ok=True)
audio_ids = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]
for audio_id in audio_ids:
    filename = f"{audio_id:02}.mp3"
    url='https://www.gutenberg.org/files/21144/mp3/21144-' + filename
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(os.path.join(output_dir, filename), "wb") as f:
            f.write(response.content)

        print(f"Descargado: {filename}")
    else:
        print(f"Error al descargar: {filename} (Código {response.status_code})")
        print(url)


Descargado: 01.mp3
Descargado: 04.mp3
Descargado: 05.mp3
Descargado: 06.mp3
Descargado: 14.mp3
Descargado: 22.mp3
Descargado: 24.mp3
Descargado: 25.mp3
Descargado: 26.mp3
Descargado: 27.mp3


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [ ]:


import openai

openai.api_key = "myKey"

def transcribe_audio(file_path):
    with open(file_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file, language="es")
    return transcript["text"].strip()

folder = r"C:\Users\silen\Downloads\audios_fabulas_mp3"
mp3_files = [f for f in os.listdir(folder) if f.endswith(".mp3")]
transcripciones = {}

for filename in mp3_files:
    file_path = os.path.join(folder, filename)
    print(f"Transcribiendo: {filename}")
    try:
        texto = transcribe_audio(file_path)
        clave = filename.replace(".mp3", "").zfill(2)
        transcripciones[f"audio{clave}"] = texto
        print(f"Texto: {texto}\n")
    except Exception as e:
        print(f"Error al transcribir {filename}:\n{e}\n")



Transcribiendo: 01.mp3
Texto: Las fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. Fin de la fábula Esta es una grabación del dominio público.

Transcribiendo: 04.mp3
Texto: Las fábulas de Esopo, grabado para LibriVox.org por Roberto Antonio Muñoz, fábula número 64, El Lobo y la Cruz. A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correra a una grulla y le pidió que le salvara de aquella situación y que enseguida le pagaría por ello. Aceptó l

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [11]:

# Incluyan a continuación todas las celdas (de código o texto) que deseen...
import json
import re
import unicodedata
for x in transcripciones:
    transcripciones[x]=re.sub(r"\d+\.\d+", "", transcripciones[x])
    transcripciones[x] = transcripciones[x].replace(".", "")
    transcripciones[x] = transcripciones[x].replace(",", "")
    transcripciones[x] = transcripciones[x].replace("!", "")
    transcripciones[x] = transcripciones[x].replace("?", "")
    transcripciones[x] = transcripciones[x].replace("¿", "")
    transcripciones[x] = transcripciones[x].replace("¡", "")
    transcripciones[x] = unicodedata.normalize("NFKD", transcripciones[x])
    transcripciones[x] = "".join(ch for ch in transcripciones[x] if not unicodedata.combining(ch))
    transcripciones[x]= transcripciones[x].replace("Sopo", "Esopo")
    transcripciones[x]= transcripciones[x].replace("Sofo", "Esopo")
    transcripciones[x] = transcripciones[x].lower()
    transcripciones[x] = transcripciones[x].lower()
    transcripciones[x]= transcripciones[x].replace("librevoxorg ", "librivoxorg")
    

for x in transcripciones:
    print(transcripciones[x])

texts = list(transcripciones.values())
inicioComun = texts[0]
for x in texts[1:]:
    while not x.startswith(inicioComun):
        inicioComun = inicioComun[:-1]

finComun = texts[0][::-1]  
for x in texts[1:]:
    x_invertido = x[::-1]
    while not x_invertido.startswith(finComun):
        finComun = finComun[:-1]
finComun = finComun[::-1]  


transcripciones_limpias = {}
for key, texto in transcripciones.items():
    if texto.startswith(inicioComun):
        texto = texto[len(inicioComun):]
    texto = re.sub(r"^.*?fabula numero\s*\d+\s*", "", texto, flags=re.IGNORECASE)
    texto = re.sub(r"\bfin\b.*", "", texto, flags=re.IGNORECASE | re.DOTALL)
    transcripciones_limpias[key] = texto.strip()
    print(f"{key}:\n{texto.strip()}\n")
nombreJson = "DatosLimpios.json"

with open(nombreJson, "w", encoding="utf-8") as file:
  json.dump(transcripciones_limpias, file, ensure_ascii=False, indent=2)

print("Gurado en Json")


file_name_json = "DatosLimpios.json"

with open(file_name_json, "r", encoding="utf-8") as file:
  DatosLimpios = json.load(file)




las fabulas de esopo grabado para librivoxorg por paulino wwwpaulinoinfo fabula numero 61 el lobo y el cordero en el templo dandose cuenta de que era perseguido por un lobo un pequeno corderito decidio refugiarse en un templo cercano lo llamo lobo y le dijo que si el sacrificador lo encontraba alli adentro lo inmolaria a su dios mejor asi replico el cordero prefiero ser victima para un dios a tener que perecer en tus colmillos si sin remedio vamos a ser sacrificados mas nos vale que sea con el mayor honor fin de la fabula esta es una grabacion del dominio publico
las fabulas de esopo grabado para librivoxorg por roberto antonio munoz fabula numero 64 el lobo y la cruz a un lobo que comia un hueso se le atraganto el hueso en la garganta y corria por todas partes en busca de auxilio encontro en su correra a una grulla y le pidio que le salvara de aquella situacion y que enseguida le pagaria por ello acepto la grulla e introdujo su cabeza en la boca del lobo sacando de la garganta el hues

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

listaTokens=[]

for x in DatosLimpios:
    tokenLimpios =[]
    z=DatosLimpios[x].split()
    z = [y for y in z if y not in stopwords.words('spanish')]
    for y in z:
         y = re.sub(r'\s{2, }', ' ', y.strip())
         tokenLimpios.append(y)
    listaTokens.append(tokenLimpios)

print(listaTokens)

    




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\silen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['lobo', 'cordero', 'templo', 'dandose', 'cuenta', 'perseguido', 'lobo', 'pequeno', 'corderito', 'decidio', 'refugiarse', 'templo', 'cercano', 'llamo', 'lobo', 'dijo', 'si', 'sacrificador', 'encontraba', 'alli', 'adentro', 'inmolaria', 'dios', 'mejor', 'asi', 'replico', 'cordero', 'prefiero', 'ser', 'victima', 'dios', 'tener', 'perecer', 'colmillos', 'si', 'remedio', 'vamos', 'ser', 'sacrificados', 'mas', 'vale', 'mayor', 'honor'], ['lobo', 'cruz', 'lobo', 'comia', 'hueso', 'atraganto', 'hueso', 'garganta', 'corria', 'todas', 'partes', 'busca', 'auxilio', 'encontro', 'correra', 'grulla', 'pidio', 'salvara', 'aquella', 'situacion', 'enseguida', 'pagaria', 'ello', 'acepto', 'grulla', 'introdujo', 'cabeza', 'boca', 'lobo', 'sacando', 'garganta', 'hueso', 'atravesado', 'pidio', 'entonces', 'cancelacion', 'paga', 'convenida', 'oye', 'amiga', 'dijo', 'lobo', 'crees', 'suficiente', 'paga', 'haber', 'sacado', 'cabeza', 'sana', 'salva', 'boca', 'nunca', 'hagas', 'favores', 'malvados', 'trafica

# **Ejercicio 4:**

In [13]:
!pip uninstall gensim -y
!pip uninstall numpy -y
!pip install gensim numpy --user

Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\site-packages\pip\_internal\cli\base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\site-packages\pip\_internal\commands\uninstall.py", line 102, in run
    uninstall_pathset.commit()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\site-packages\pip\_internal\req\req_uninstall.py", line 420, in commit
    self._moved_paths.commit()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\site-packages\pip\_internal\req\req_uninstall.py", line 273, in commit
    save_dir.cleanup()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\site

You should consider upgrading via the 'C:\Users\silen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



  Using cached gensim-4.3.3-cp39-cp39-win_amd64.whl (24.0 MB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)


In [14]:
import gensim
import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel
palabrasClave=[]
for x in listaTokens:
   
    dicCuentos = corpora.Dictionary(listaTokens)
    dicCuentos.filter_extremes(no_below=2, no_above=0.5)
    corpus = [dicCuentos.doc2bow(text) for text in listaTokens]
    
    for i, doc in enumerate(corpus[:10]):
        print(f"Cuento {i+1}:")
        for word_id, freq in doc:
            print(f"  {dicCuentos[word_id]}: {freq}")


    lda_model = gensim.models.LdaModel(corpus, 
                                    num_topics=1,  # Cambia este número según cuántos temas quieras
                                    id2word=dicCuentos,
                                    update_every=1, 
                                    passes=18, 
                                    random_state=42)

    key_words = lda_model.show_topic(topicid=0, topn=30)
    palabrasClave.append(key_words)
    for word, prob in key_words:
        print(f"{word} ({prob:.2f}%)")
   


Cuento 1:
  lobo: 3
  mas: 1
  mayor: 1
  mejor: 1
  replico: 1
  si: 2
  tener: 1
Cuento 2:
  lobo: 4
  si: 1
  encontro: 1
  nunca: 1
  partes: 1
  pues: 1
Cuento 3:
  lobo: 2
  mas: 1
  mejor: 1
  si: 1
  encontro: 1
  amigo: 1
  camino: 1
  habia: 2
  llevo: 1
  lobos: 1
  sino: 1
  vez: 1
Cuento 4:
  lobo: 3
  si: 1
  tener: 1
  partes: 1
  lobos: 1
  vez: 1
  ahi: 1
  casa: 1
  poder: 1
  primero: 1
Cuento 5:
  lobo: 3
  replico: 1
  sino: 2
  casa: 1
  infeliz: 1
  vio: 1
Cuento 6:
  nunca: 1
  bien: 1
  perro: 2
Cuento 7:
  mayor: 1
  nunca: 1
  pues: 1
  llevo: 1
  vio: 1
  bien: 1
  perro: 3
  carne: 2
  solo: 1
  trozo: 2
Cuento 8:
  amigo: 1
  ahi: 1
  poder: 1
  perro: 2
  carne: 1
  trozo: 1
  volvio: 1
Cuento 9:
  mas: 1
  si: 1
  amigo: 1
  habia: 1
  sino: 1
  perro: 2
  solo: 1
  mismo: 1
Cuento 10:
  encontro: 1
  camino: 1
  casa: 1
  primero: 1
  infeliz: 1
  vio: 1
  perro: 4
  volvio: 1
  mismo: 1
lobo (0.12%)
perro (0.11%)
si (0.05%)
sino (0.04%)
mas (0.03%)
car

# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [17]:
fabulas_texto = [" ".join(tokens) for tokens in listaTokens]
resumenes = []

for i, fabula in enumerate(fabulas_texto):
    prompt = f"""
    A continuación se presenta una fábula en forma de texto limpio:

    "{fabula}"

    Por favor:
    1. Resume la fábula en un solo oracion que de una descripcion del tema pricipal estas son palabras extraidas por LDA de algunas fabulas  escritas por Esopo. En la fabula 8 el perro no es carnicero solo un ladron 
    2. Genera tres enunciados distintos que describan partes o momentos diferentes de la historia.

    Devuelve el resultado con este formato:
    - Resumen: ...
    - Enunciado 1: ...
    - Enunciado 2: ...
    - Enunciado 3: ...
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",  # O "gpt-3.5-turbo"
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=300
    )
    
    texto_generado = response['choices'][0]['message']['content']
    resumenes.append({
        "fabula": fabula,
        "resumen_y_enunciados": texto_generado
    })

# Mostrar los resultados
for i, r in enumerate(resumenes):
    print(f"\n Fábula {i+1}:")
    print(r["resumen_y_enunciados"])





 Fábula 1:
- Resumen: La fábula trata sobre un cordero que, al ser perseguido por un lobo, decide refugiarse en un templo, prefiriendo ser sacrificado por un dios antes que caer en las garras del lobo.
- Enunciado 1: Un pequeño corderito se da cuenta que está siendo perseguido por un lobo y decide buscar refugio en un templo cercano.
- Enunciado 2: El lobo intenta convencer al cordero para que salga del templo, advirtiéndole que si el sacrificador lo encuentra allí, lo inmolará a un dios.
- Enunciado 3: Sin embargo, el cordero responde que prefiere ser la víctima de un dios que perecer a manos del lobo, argumentando que si de todos modos van a ser sacrificados, es preferible que se haga con mayor honor.

 Fábula 2:
- Resumen: La fábula narra cómo un lobo, tras atragantarse con un hueso, pide ayuda a un grulla, prometiendo pagarle por su auxilio, pero una vez salvado, se niega a cumplir su promesa argumentando que el hecho de que la grulla haya podido sacar su cabeza de su boca sin ser

# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



None

# **Fin de la actividad LDA y LMM: audio-a-texto**